In [2]:
import os
import sys
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

In [4]:
parameter="_S2_"
biopara='FAPAR'
inputdir="D:\\vegetation\\gee\\FAPAR"
df_list=[]
tile_list=[]

key1='S2'+'MODIS'+'_'+'S2_'+biopara ## example: 'S2MODIS_LAI'      
key2='S2'+'MODIS'+'_'+biopara+'_A'  ## example 'S2MODIS_LAI_A'      
key3='S2'+'MODIS'+'_'+biopara+'_P'  ## example 'S2MODIS_LAI_P'
key4='S2'+'MODIS'+'_'+biopara+'_U'  ## example 'S2MODIS_LAI_U'
key5='S2'+'CGLS'+'_'+'S2_'+biopara  ## example: 'S2CGLS_LAI'
key6='S2'+'CGLS'+'_'+biopara+'_A'  ## example 'S2CGLS_LAI_A'
key7='S2'+'CGLS'+'_'+biopara+'_P'  ## example 'S2CGLS_LAI_P'
key8='S2'+'CGLS'+'_'+biopara+'_U'  ## example 'S2CGLS_LAI_U'

column_names = ['Tile','Ezone','NumPoints', 'Needle', 'Broadleaf', 'Mixed', key1, key2, key3, key4, key5, key6, key7, key8]
tile_df = pd.DataFrame(columns = column_names)
### get the filenames of all CSV files
for filename in os.listdir(inputdir):
      if filename.find(parameter)!=-1: 
           # print(filename)
            df_list.append(filename.split('_')[0])  ## tilename located at the beginning of filename
#print (df_list)  ## get a list of tiles
unique = [tile_list.append(x) for x in df_list if x not in tile_list] # get unique list of tiles, stored at tile_list 
print (tile_list)
### loop through each tile
for tile in tile_list:
        ## processing S2-MODIS file fist
        sensor='MODIS'
        scale=10.0
        filenameMODIS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
        df=pd.read_csv (inputdir+"\\"+filenameMODIS, index_col=0)        
        df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/500  ### MODIS-S2; change S2 scale from 1000 to 500 

        MODIS_num=len(df)
        MODIS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/MODIS_num  ## needle  lc = [1.0, 2.0] 
        MODIS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/MODIS_num
        MODIS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/MODIS_num
        MODIS_ezones=df['S2'+sensor+'_EZ'].unique() # get unique ezone from ez column
      #  key1='S2'+sensor+'_'+biopara ### example: S2MODIS_LAI
        S2MODIS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2MODIS for LAI or FAPAR or FCOVER
      #  key2='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_MODIS_A'
        MODIS_bias=df[sensor+'S2_'+'diff'].mean() ### accuracy, bias
       # key3='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_MODIS_P'
        MODIS_precision=((df[sensor+'S2_'+'diff']-MODIS_bias)** 2).mean() **.5 ### precision
      #  key4='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_MODIS_P'
        MODIS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5 ### uncertainty
        #print (tile, MODIS_ezones, MODIS_num, MODIS_needle, MODIS_broadleaf, MODIS_mixed, MODIS_bias, MODIS_precision, MODIS_rmse)
        
        ## processing S2-CGLS file 
        sensor='CGLS'
        scale=30.0      
        filenameCGLS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
        df=pd.read_csv (inputdir+"\\"+filenameCGLS, index_col=0)        
                
        df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/500   ### CGLS-S2: change S2 scale from 1000 to 500 
        CGLS_num=len(df)
        CGLS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/CGLS_num  ## needle  lc = [1.0, 2.0] 
        CGLS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/CGLS_num
        CGLS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/CGLS_num
        CGLS_ezones=df['S2'+sensor+'_EZ'].unique()
     #   key5='S2'+sensor+'_'+biopara  ### example: S2CGLS_LAI
        S2CGLS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2CGLS for LAI or FAPAR or FCOVER
        
     #   key6='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_CGLS_A'
        CGLS_bias=df[sensor+'S2_'+'diff'].mean()  ### accuracy, bias
     #   key7='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_CGLS_P'
        CGLS_precision=((df[sensor+'S2_'+'diff']-CGLS_bias)** 2).mean() **.5 ### precision
     #   key8='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_CGLS_U'
        CGLS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5   ### uncertainty
        
        #print (tile, CGLS_ezones, CGLS_num, CGLS_needle, CGLS_broadleaf, CGLS_mixed, CGLS_bias, CGLS_precision, CGLS_rmse)
        row= {'Tile':tile,'Ezone':MODIS_ezones,'NumPoints':MODIS_num, 'Needle':MODIS_needle, 'Broadleaf':MODIS_broadleaf, 'Mixed':MODIS_mixed, \
              key1:S2MODIS, key2:MODIS_bias,key3:MODIS_precision,key4:MODIS_rmse,key5:S2CGLS,key6:CGLS_bias,key7:CGLS_precision,key8:CGLS_rmse}    
        ## add each tile stats to tile dataframe
        tile_df=tile_df.append(row, ignore_index=True)      
tile_df.to_csv(inputdir+"\\"+biopara+'_Tile_Stas_Table1.csv')

['08WNU', '10UCD', '10VCP', '11VNG', '14UPD', '15VVD', '18UUU', '20ULU', '20UQD']


In [5]:

#def calStas(para, inputdir, dominantLC, dominantEZ, countthres):
def caltileStat(biopara, inputdir):

    parameter="_S2_"
    #biopara='FAPAR'
    #inputdir="D:\\vegetation\\gee\\FAPAR"
    df_list=[]
    tile_list=[]

    key1='S2'+'MODIS'+'_'+'S2_'+biopara ## example: 'S2MODIS_LAI'      
    key2='S2'+'MODIS'+'_'+biopara+'_A'  ## example 'S2MODIS_LAI_A'      
    key3='S2'+'MODIS'+'_'+biopara+'_P'  ## example 'S2MODIS_LAI_P'
    key4='S2'+'MODIS'+'_'+biopara+'_U'  ## example 'S2MODIS_LAI_U'
    key5='S2'+'MODIS'+'_'+biopara+'_R2' ## example 'S2MODIS_LAI_R2'
    key6='S2'+'MODIS'+'_'+biopara+'_SLOPE' ## example 'S2MODIS_LAI_SLOPE'
    
    key7='S2'+'CGLS'+'_'+'S2_'+biopara  ## example: 'S2CGLS_LAI'
    key8='S2'+'CGLS'+'_'+biopara+'_A'  ## example 'S2CGLS_LAI_A'
    key9='S2'+'CGLS'+'_'+biopara+'_P'  ## example 'S2CGLS_LAI_P'
    key10='S2'+'CGLS'+'_'+biopara+'_U'  ## example 'S2CGLS_LAI_U'
    key11='S2'+'CGLS'+'_'+biopara+'_R2' ## example 'S2CGLS_LAI_R2'
    key12='S2'+'CGLS'+'_'+biopara+'_SLOPE'## example 'S2CGLS_LAI_SLOPE'
    
    column_names = ['Tile','Ezone','NumPoints', 'Needle', 'Broadleaf', 'Mixed', key1, key2, key3, key4, key5, key6, key7, key8, key9,key10, key11,key12]
    tile_df = pd.DataFrame(columns = column_names)
    ### get the filenames of all CSV files
    for filename in os.listdir(inputdir):
          if filename.find(parameter)!=-1: 
               # print(filename)
                df_list.append(filename.split('_')[0])  ## tilename located at the beginning of filename
    #print (df_list)  ## get a list of tiles
    unique = [tile_list.append(x) for x in df_list if x not in tile_list] # get unique list of tiles, stored at tile_list 
    print (tile_list)
    ### loop through each tile
    for tile in tile_list:
            ## processing S2-MODIS file fist
            if biopara!='FCOVER':
                sensor='MODIS'
                if biopara=='LAI': 
                    scale=10.0
                elif biopara=='FAPAR':
                    scale=100
                else:
                    scale='none'  ### need updated later

                filenameMODIS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
                df=pd.read_csv (inputdir+"\\"+filenameMODIS, index_col=0)        
                df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/1000  ### MODIS-S2; change S2 scale from 1000 to 500 

                MODIS_num=len(df)
                MODIS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/MODIS_num  ## needle  lc = [1.0, 2.0] 
                MODIS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/MODIS_num
                MODIS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/MODIS_num
                MODIS_ezones=df['S2'+sensor+'_EZ'].unique() # get unique ezone from ez column
              #  key1='S2'+sensor+'_'+biopara ### example: S2MODIS_LAI
                S2MODIS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2MODIS for LAI or FAPAR or FCOVER
              #  key2='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_MODIS_A'
                MODIS_bias=df[sensor+'S2_'+'diff'].mean() ### accuracy, bias
               # key3='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_MODIS_P'
                MODIS_precision=((df[sensor+'S2_'+'diff']-MODIS_bias)** 2).mean() **.5 ### precision
              #  key4='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_MODIS_P'
                MODIS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5 ### uncertainty
                #print (tile, MODIS_ezones, MODIS_num, MODIS_needle, MODIS_broadleaf, MODIS_mixed, MODIS_bias, MODIS_precision, MODIS_rmse)

                ### convert one column to numpy array
                x=(df[sensor+'_'+biopara]/scale).to_numpy()
                y=(df['S2'+sensor+'_'+biopara]/1000).to_numpy()
                 ##Calculate a linear least-squares regression for two sets of measurements.
                lsq_res=stats.linregress(x,y)
                print ('linear estimator:modis', lsq_res)
                MODIS_R2=lsq_res.rvalue**2
                MODIS_SLOPE=lsq_res.slope      

            ## processing S2-CGLS file 
            sensor='CGLS'
            if biopara=='LAI': 
                scale=30.0
            elif biopara=='FAPAR':
                scale=250
            else:
                scale=250
                 
            filenameCGLS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
            df=pd.read_csv (inputdir+"\\"+filenameCGLS, index_col=0)        

            df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/1000   ### CGLS-S2: change S2 scale from 1000 to 500 
            CGLS_num=len(df)
            CGLS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/CGLS_num  ## needle  lc = [1.0, 2.0] 
            CGLS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/CGLS_num
            CGLS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/CGLS_num
            CGLS_ezones=df['S2'+sensor+'_EZ'].unique()
         #   key5='S2'+sensor+'_'+biopara  ### example: S2CGLS_LAI
            S2CGLS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2CGLS for LAI or FAPAR or FCOVER

         #   key6='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_CGLS_A'
            CGLS_bias=df[sensor+'S2_'+'diff'].mean()  ### accuracy, bias
         #   key7='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_CGLS_P'
            CGLS_precision=((df[sensor+'S2_'+'diff']-CGLS_bias)** 2).mean() **.5 ### precision
         #   key8='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_CGLS_U'
            CGLS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5   ### uncertainty
            
            ### convert one column to numpy array
            x=(df[sensor+'_'+biopara]/scale).to_numpy()
            y=(df['S2'+sensor+'_'+biopara]/1000).to_numpy()
             ##Calculate a linear least-squares regression for two sets of measurements.
            lsq_res=stats.linregress(x,y)
            print ('linear estimator:CGLS', lsq_res)
            CGLS_R2=lsq_res.rvalue**2
            CGLS_SLOPE=lsq_res.slope           
            

            #print (tile, CGLS_ezones, CGLS_num, CGLS_needle, CGLS_broadleaf, CGLS_mixed, CGLS_bias, CGLS_precision, CGLS_rmse)
            row= {'Tile':tile,'Ezone':MODIS_ezones,'NumPoints':MODIS_num, 'Needle':MODIS_needle, 'Broadleaf':MODIS_broadleaf, 'Mixed':MODIS_mixed, \
                  key1:S2MODIS, key2:MODIS_bias,key3:MODIS_precision,key4:MODIS_rmse,key5:MODIS_R2,key6:MODIS_SLOPE,
                  key7:S2CGLS,key8:CGLS_bias,key9:CGLS_precision,key10:CGLS_rmse,key11:CGLS_R2,key12:CGLS_SLOPE}    
            ## add each tile stats to tile dataframe
            tile_df=tile_df.append(row, ignore_index=True)      
    tile_df.to_csv(inputdir+"\\"+biopara+'_Tile_Stas_Table1.csv')

In [8]:
#def calStas(para, inputdir, dominantLC, dominantEZ, countthres):
def caltileStat2(biopara, inputdir):

    parameter="_S2_"
    #biopara='FAPAR'
    #inputdir="D:\\vegetation\\gee\\FAPAR"
    df_list=[]
    tile_list=[]

    key1='S2'+'MODIS'+'_'+'S2_'+biopara ## example: 'S2MODIS_LAI'      
    key2='S2'+'MODIS'+'_'+biopara+'_A'  ## example 'S2MODIS_LAI_A'      
    key3='S2'+'MODIS'+'_'+biopara+'_P'  ## example 'S2MODIS_LAI_P'
    key4='S2'+'MODIS'+'_'+biopara+'_U'  ## example 'S2MODIS_LAI_U'
    key5='S2'+'MODIS'+'_'+biopara+'_R2' ## example 'S2MODIS_LAI_R2'
    key6='S2'+'MODIS'+'_'+biopara+'_SLOPE' ## example 'S2MODIS_LAI_SLOPE'
    
    key7='S2'+'CGLS'+'_'+'S2_'+biopara  ## example: 'S2CGLS_LAI'
    key8='S2'+'CGLS'+'_'+biopara+'_A'  ## example 'S2CGLS_LAI_A'
    key9='S2'+'CGLS'+'_'+biopara+'_P'  ## example 'S2CGLS_LAI_P'
    key10='S2'+'CGLS'+'_'+biopara+'_U'  ## example 'S2CGLS_LAI_U'
    key11='S2'+'CGLS'+'_'+biopara+'_R2' ## example 'S2CGLS_LAI_R2'
    key12='S2'+'CGLS'+'_'+biopara+'_SLOPE'## example 'S2CGLS_LAI_SLOPE'
    
    if biopara!='FCOVER':
        column_names = ['Tile','Ezone','NumPoints', 'Needle', 'Broadleaf', 'Mixed', key1, key2, key3, key4, key5, key6, key7, key8, key9,key10, key11,key12]
        tile_df = pd.DataFrame(columns = column_names)
    else:
        column_names = ['Tile','Ezone','NumPoints', 'Needle', 'Broadleaf', 'Mixed',  key7, key8, key9,key10, key11,key12]
        tile_df = pd.DataFrame(columns = column_names)
    ### get the filenames of all CSV files
    for filename in os.listdir(inputdir):
          if filename.find(parameter)!=-1: 
               # print(filename)
                df_list.append(filename.split('_')[0])  ## tilename located at the beginning of filename
    #print (df_list)  ## get a list of tiles
    unique = [tile_list.append(x) for x in df_list if x not in tile_list] # get unique list of tiles, stored at tile_list 
    print (tile_list)
    ### loop through each tile
    for tile in tile_list:
        ## processing S2-CGLS file 
            sensor='CGLS'
            if biopara=='LAI': 
                scale=30.0
            elif biopara=='FAPAR':
                scale=250
            else:
                scale=250
                 
            filenameCGLS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
            df=pd.read_csv (inputdir+"\\"+filenameCGLS, index_col=0)        

            df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/1000   ### CGLS-S2: change S2 scale from 1000 to 500 
            CGLS_num=len(df)
            CGLS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/CGLS_num  ## needle  lc = [1.0, 2.0] 
            CGLS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/CGLS_num
            CGLS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/CGLS_num
            CGLS_ezones=df['S2'+sensor+'_EZ'].unique()
         #   key5='S2'+sensor+'_'+biopara  ### example: S2CGLS_LAI
            S2CGLS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2CGLS for LAI or FAPAR or FCOVER

         #   key6='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_CGLS_A'
            CGLS_bias=df[sensor+'S2_'+'diff'].mean()  ### accuracy, bias
         #   key7='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_CGLS_P'
            CGLS_precision=((df[sensor+'S2_'+'diff']-CGLS_bias)** 2).mean() **.5 ### precision
         #   key8='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_CGLS_U'
            CGLS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5   ### uncertainty
            
            ### convert one column to numpy array
            x=(df[sensor+'_'+biopara]/scale).to_numpy()
            y=(df['S2'+sensor+'_'+biopara]/1000).to_numpy()
             ##Calculate a linear least-squares regression for two sets of measurements.
            lsq_res=stats.linregress(x,y)
            print ('linear estimator:CGLS', lsq_res)
            CGLS_R2=lsq_res.rvalue**2
            CGLS_SLOPE=lsq_res.slope           
            

             ## processing S2-MODIS file 
            if biopara!='FCOVER':
                sensor='MODIS'
                if biopara=='LAI': 
                    scale=10.0
                elif biopara=='FAPAR':
                    scale=100
                else:
                    scale='none'  ### need updated later

                filenameMODIS=tile+'_'+biopara+'_'+'S2'+'_'+sensor+'.csv'
                df=pd.read_csv (inputdir+"\\"+filenameMODIS, index_col=0)        
                df[sensor+'S2_'+'diff']=df[sensor+'_'+biopara]/scale-df['S2'+sensor+'_'+biopara]/1000  ### MODIS-S2; change S2 scale from 1000 to 500 

                MODIS_num=len(df)
                MODIS_needle=len(df.loc[df['S2'+sensor+'_LC'].isin([1.0,2.0])])/MODIS_num  ## needle  lc = [1.0, 2.0] 
                MODIS_broadleaf=len(df.loc[df['S2'+sensor+'_LC'].isin([3.0,4.0,5.0])])/MODIS_num
                MODIS_mixed=len(df.loc[df['S2'+sensor+'_LC'].isin([6.0])])/MODIS_num
                MODIS_ezones=df['S2'+sensor+'_EZ'].unique() # get unique ezone from ez column
              #  key1='S2'+sensor+'_'+biopara ### example: S2MODIS_LAI
                S2MODIS=df['S2'+sensor+'_'+biopara].mean()/1000  ### S2MODIS for LAI or FAPAR or FCOVER
              #  key2='S2'+sensor+'_'+biopara+'_A'  ## example 'LAI_MODIS_A'
                MODIS_bias=df[sensor+'S2_'+'diff'].mean() ### accuracy, bias
               # key3='S2'+sensor+'_'+biopara+'_P'  ## example 'LAI_MODIS_P'
                MODIS_precision=((df[sensor+'S2_'+'diff']-MODIS_bias)** 2).mean() **.5 ### precision
              #  key4='S2'+sensor+'_'+biopara+'_U'  ## example 'LAI_MODIS_P'
                MODIS_rmse=((df[sensor+'S2_'+'diff']) ** 2).mean() ** .5 ### uncertainty
                #print (tile, MODIS_ezones, MODIS_num, MODIS_needle, MODIS_broadleaf, MODIS_mixed, MODIS_bias, MODIS_precision, MODIS_rmse)

                ### convert one column to numpy array
                x=(df[sensor+'_'+biopara]/scale).to_numpy()
                y=(df['S2'+sensor+'_'+biopara]/1000).to_numpy()
                 ##Calculate a linear least-squares regression for two sets of measurements.
                lsq_res=stats.linregress(x,y)
                print ('linear estimator:modis', lsq_res)
                MODIS_R2=lsq_res.rvalue**2
                MODIS_SLOPE=lsq_res.slope      
            #print (tile, CGLS_ezones, CGLS_num, CGLS_needle, CGLS_broadleaf, CGLS_mixed, CGLS_bias, CGLS_precision, CGLS_rmse)
                row= {'Tile':tile,'Ezone':MODIS_ezones,'NumPoints':MODIS_num, 'Needle':MODIS_needle, 'Broadleaf':MODIS_broadleaf, 'Mixed':MODIS_mixed, \
                      key1:S2MODIS, key2:MODIS_bias,key3:MODIS_precision,key4:MODIS_rmse,key5:MODIS_R2,key6:MODIS_SLOPE,
                      key7:S2CGLS,key8:CGLS_bias,key9:CGLS_precision,key10:CGLS_rmse,key11:CGLS_R2,key12:CGLS_SLOPE} 
            else:
                row= {'Tile':tile,'Ezone':CGLS_ezones,'NumPoints':CGLS_num, 'Needle':CGLS_needle, 'Broadleaf':CGLS_broadleaf, 'Mixed':CGLS_mixed, \
                      key7:S2CGLS,key8:CGLS_bias,key9:CGLS_precision,key10:CGLS_rmse,key11:CGLS_R2,key12:CGLS_SLOPE} 
                ## add each tile stats to tile dataframe
            tile_df=tile_df.append(row, ignore_index=True)      
    tile_df.to_csv(inputdir+"\\"+biopara+'_Tile_Stas_Table1.csv')

In [10]:
caltileStat2('FAPAR', 'D:\\vegetation\\gee\\FAPAR')

['05WNM', '06WWV', '08WNU', '09VXG', '09VXH', '10TDN', '10UCD', '10VCP', '10VFP', '10VFQ', '11TNN', '11VLD', '11VLE', '11VNG', '13TCE', '13VEC', '14SQA', '14UPD', '15SVV', '15SWR', '15VVD', '16SDF', '16UEB', '17RKP', '17SMD', '17SPT', '17TLH', '18TTM', '18TVP', '18UUU', '19TDM', '19TFN', '19UES', '20ULU', '20UQD']
linear estimator:CGLS LinregressResult(slope=0.7937532052391401, intercept=-0.10688719758974985, rvalue=0.7813892767015639, pvalue=0.0, stderr=0.0023339436008654444)
linear estimator:modis LinregressResult(slope=0.43333277525263525, intercept=0.14740752827336567, rvalue=0.5870620166436772, pvalue=0.0, stderr=0.00117792654845348)
linear estimator:CGLS LinregressResult(slope=0.7640565854125644, intercept=-0.09245982947197484, rvalue=0.8606779195042996, pvalue=0.0, stderr=0.001505685629972665)
linear estimator:modis LinregressResult(slope=0.5097924794921012, intercept=0.07845661757512168, rvalue=0.7524783195961242, pvalue=0.0, stderr=0.000954399387040378)
linear estimator:CGLS L

In [11]:
caltileStat2('LAI', 'D:\\vegetation\\gee\\LAI')

['05WNM', '06WWV', '08WNU', '09VXG', '09VXH', '10TDN', '10UCD', '10VCP', '10VFP', '10VFQ', '11TNN', '11VLD', '11VLE', '11VNG', '13TCE', '13VEC', '14SQA', '14UPD', '15SVV', '15SWR', '15VVD', '16SDF', '16UEB', '17RKP', '17SMD', '17SPT', '17TLH', '18TTM', '18TVP', '18UUU', '19TDM', '19TFN', '19UES', '20ULU', '20UQD']
linear estimator:CGLS LinregressResult(slope=0.34267712368101705, intercept=0.7665880048741243, rvalue=0.7961404140426196, pvalue=0.0, stderr=0.0009589656183966354)
linear estimator:modis LinregressResult(slope=0.2982097124370275, intercept=1.0061914961850007, rvalue=0.6006971875930781, pvalue=0.0, stderr=0.0007824813792183607)
linear estimator:CGLS LinregressResult(slope=0.4173806221326319, intercept=0.604855319366116, rvalue=0.7731090433947023, pvalue=0.0, stderr=0.0011407070351571718)
linear estimator:modis LinregressResult(slope=0.3604774154305793, intercept=0.7913069063898388, rvalue=0.6328082276498226, pvalue=0.0, stderr=0.000943447577623417)
linear estimator:CGLS Linre

In [9]:
caltileStat2('FCOVER', 'D:\\vegetation\\gee\\FCOVER')

['05WNM', '06WWV', '08WNU', '09VXG', '09VXH', '10TDN', '10UCD', '10VCP', '10VFP', '10VFQ', '11TNN', '11VLD', '11VLE', '11VNG', '13TCE', '13VEC', '14SQA', '14UPD', '15SVV', '15SWR', '15VVD', '16SDF', '16UEB', '17RKP', '17SMD', '17SPT', '17TLH', '18TTM', '18TVP', '18UUU', '19TDM', '19TFN', '19UES', '20ULU', '20UQD']
linear estimator:CGLS LinregressResult(slope=0.5167538605383143, intercept=0.12779601508150912, rvalue=0.7581926966657103, pvalue=0.0, stderr=0.0016361730709865488)
linear estimator:CGLS LinregressResult(slope=0.5147657406310099, intercept=0.09898143394139586, rvalue=0.8465403797692602, pvalue=0.0, stderr=0.0010783066587483725)
linear estimator:CGLS LinregressResult(slope=0.6278394090853041, intercept=0.05622146411458595, rvalue=0.7579731168005133, pvalue=0.0, stderr=0.0018191101953389118)
linear estimator:CGLS LinregressResult(slope=0.5617745201357707, intercept=0.10159497319008454, rvalue=0.7601679090640976, pvalue=0.0, stderr=0.0015757763813331195)
linear estimator:CGLS Li

In [12]:
caltileStat2('LAI', 'D:\\vegetation\\gee\\LAI_CCRS')

['07VDL', '09VVG', '09WXP', '10TFP', '10UGV', '10VEL', '11TPM', '11UNA', '12RUV', '13SCT', '13VDH', '13WDR', '14UPE', '15SUR', '15SWR', '15TUN', '15TYL', '16UEE', '17SMU', '18TVP', '18UVA', '18WWT', '18WXV', '19TFM', '19VED', '21TYN', '21UUA', '21UWQ']
linear estimator:CGLS LinregressResult(slope=0.26539742602594363, intercept=0.8967423278418815, rvalue=0.5970385743235856, pvalue=0.0, stderr=0.0010498151646577726)
linear estimator:modis LinregressResult(slope=0.20151488076189322, intercept=1.1614191823047033, rvalue=0.4406985727226669, pvalue=0.0, stderr=0.0007335575045419229)
linear estimator:CGLS LinregressResult(slope=0.48370069540204125, intercept=0.5732797204048325, rvalue=0.694525452207268, pvalue=0.0, stderr=0.0017147936283816475)
linear estimator:modis LinregressResult(slope=0.2762595137478765, intercept=1.0673598197053789, rvalue=0.44184314496620725, pvalue=0.0, stderr=0.0012567162598211027)
linear estimator:CGLS LinregressResult(slope=0.353677757921127, intercept=0.6639879535

In [ ]:
caltileStat2('FAPAR', 'D:\\vegetation\\gee\\FAPAR_CCRS')